In [410]:
#Import the libraries I'll use.
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import re

In [542]:
#Starting by looking at a specific game page and looking what class is the box info.
r = requests.get("https://en.wikipedia.org/wiki/The_Oregon_Trail_(1971_video_game)")
soups = bs(r.content,"lxml")

info_box = soups.find(class_="infobox hproduct")
info_rows = info_box.find_all("tr")
for row in info_rows:
    print(row.prettify())

<tr>
 <th class="infobox-above fn" colspan="2" style="font-size:125%;font-style:italic;">
  The Oregon Trail
 </th>
</tr>

<tr>
 <td class="infobox-image" colspan="2">
  <a class="image" href="/wiki/File:TheOregonTrail1971Gameplay.png">
   <img alt="TheOregonTrail1971Gameplay.png" data-file-height="180" data-file-width="552" decoding="async" height="82" src="//upload.wikimedia.org/wikipedia/en/thumb/2/26/TheOregonTrail1971Gameplay.png/250px-TheOregonTrail1971Gameplay.png" srcset="//upload.wikimedia.org/wikipedia/en/thumb/2/26/TheOregonTrail1971Gameplay.png/375px-TheOregonTrail1971Gameplay.png 1.5x, //upload.wikimedia.org/wikipedia/en/thumb/2/26/TheOregonTrail1971Gameplay.png/500px-TheOregonTrail1971Gameplay.png 2x" width="250"/>
  </a>
  <div class="infobox-caption" style="font-size: 95%;">
   Picture of
   <a href="/wiki/Teleprinter" title="Teleprinter">
    teleprinter
   </a>
   output of the game, featuring a single round of gameplay
  </div>
 </td>
</tr>

<tr>
 <th class="infobox-

In [543]:
#Fix linebreaks and specific release date.
for br in soups.find_all("br"):
    br.replace_with(" ")

lst = []
for t in info_box.find_all("li"):
    lst.append(t.get_text())
    
print(lst)

['NA: December 3, 1971', 'NA: 1975']


In [544]:
def get_content_value(row_data):
    if row_data.find("ui"):
        return [li.get_text() for li in row_data.find_all("li")]
    else:
        return row_data.get_text()

#Creating a dictionary to use the keys as categories and the values of each category.
games_info = {}
for index, row in enumerate(info_rows):
    if index == 0:
        games_info["Title"] = row.find("th").get_text()
    elif index == 1: # That is the picture, I won't use it.
        continue 
    else:
        key = row.find("th").get_text()
        value = row.find("td").get_text()
        games_info[key] = value

games_info["Release"] = lst

games_info

{'Title': 'The Oregon Trail',
 'Developer(s)': 'Don Rawitsch, Bill Heinemann, and Paul Dillenberger',
 'Publisher(s)': 'MECC',
 'Series': 'The Oregon Trail',
 'Platform(s)': 'Mainframe (HP 2100, CDC Cyber 70/73-26) PC (Apple II, Atari 8-Bit, Commodore 64)',
 'Release': ['NA: December 3, 1971', 'NA: 1975'],
 'Genre(s)': 'Strategy',
 'Mode(s)': 'Single-player'}

In [549]:
#Getting the table of all the games list
games = requests.get("https://en.wikipedia.org/wiki/List_of_video_games_considered_the_best")
soup_game = bs(games.content,"lxml")
all_games_lst = soup_game.select(".wikitable.sortable i")

In [613]:
#Looking at the first 10 game to see if it worked.
all_games_lst[:10]

'/wiki/The_Oregon_Trail_(1971_video_game)'

In [644]:
#Created a function to look at list elements and strip the whirespaces and seperate them by a single one space
# and put them under a list format.
def get_content_value(row_data):
    if row_data.find("li"):
        return [li.get_text(" ",strip=True) for li in row_data.find_all("li")]
    else:
        return row_data.get_text()

def clean_refs(soup):
    desired_string = ''.join([x.strip() for x in str(soup.title).split('\r\n')])

#Created a function to take each url and the information about each game.
def get_info_box(url):
    r = requests.get(url)
    soups = bs(r.content,"lxml")
    info_box = soups.find(class_="infobox hproduct")
    info_rows = info_box.find_all("tr")
    
    clean_refs(soups)
    
    games_info = {}
    for index, row in enumerate(info_rows):
        if index == 0:
            games_info["Title"] = row.find("th").get_text()
        elif index == 1: # That is the picture, I won't use it.
            continue 
        else:
            key = row.find("th").get_text()
            value = row.find("td").get_text()
            games_info[key] = value
            
    return games_info

In [645]:
get_info_box("https://en.wikipedia.org/wiki/Nights_into_Dreams")

{'Title': 'NiGHTS into dreams...',
 'Developer(s)': 'Sonic Team',
 'Publisher(s)': 'Sega',
 'Director(s)': 'Naoto Ohshima',
 'Producer(s)': 'Yuji Naka',
 'Designer(s)': 'Takashi Iizuka',
 'Programmer(s)': 'Yuji Naka',
 'Artist(s)': '\nKazuyuki Hoshino\nNaoto Ohshima\n',
 'Composer(s)': '\nNaofumi Hataya\nTomoko Sasaki\nFumie Kumatani\n',
 'Series': 'NiGHTS\xa0',
 'Platform(s)': '\nSega Saturn\nPlayStation 2\nMicrosoft Windows\nPlayStation 3\nXbox 360\n',
 'Release': '\n5 July 1996\n\nSega SaturnJP: 5 July 1996NA: 20 August 1996EU: 7 October 1996\nPlayStation 2JP: 21 February 2008\nMicrosoft WindowsWW: 2 October 2012\nPlayStation 3NA: 2 October 2012EU: 3 October 2012JP: 4 October 2012\nXbox 360WW: 5 October 2012\n',
 'Genre(s)': 'Action',
 'Mode(s)': 'Single-player, multiplayer'}

In [615]:
#Loaded the website I'm going to get all the video games list and iterate on each game to get the title and
#the link to the wikipedia game page.
r = requests.get("https://en.wikipedia.org/wiki/List_of_video_games_considered_the_best")
soup_game = bs(r.content,"lxml")
games = soup_game.select(".wikitable.sortable i a")

wiki_path = "https://en.wikipedia.org"

game_info_list = []
for index, game in enumerate(games):
    try:
        relative_path = game["href"]
        full_wiki_path = wiki_path+relative_path
        title = game["title"]
        game_info_list.append(get_info_box(full_wiki_path))
        
    except Exception: #3 games won't open so I decided to pass them.
        pass
        

In [616]:
len(game_info_list)

236

In [617]:
import json

def save(title, data):
    with open(title, "w", encoding="utf8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

In [619]:
import json

def load(title):
    with open(title, "r", encoding="utf8") as f:
        return json.load(f)

In [620]:
#Save the dictionary of the games as a json file.
save("Video Games Data.json", game_info_list)

In [621]:
#Reload the data so I won't have to do requests again.
game_info_list = load("Video Games Data.json")

# Clean The Data:

### Remove references box from the data and the keys.
### remove whitespaces from the data.
### Replcae "\n" with an empty string.
### Remove (s) from the dictionary keys, special sign such as "./!" etc and strip whitespaces as well.
### Fix release dates.
### 

In [624]:
#Clean up references (remove [1] etc.).


AttributeError: 'dict' object has no attribute 'replace'